In [10]:
import torch
import logging


In [11]:

def initialize_model(model_name, num_classes, pretrained=True):
    """
    Initialize a pre-trained model.
    """
    if model_name == "efficientnet":
        model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=pretrained)
        model.classifier.fc = torch.nn.Linear(model.classifier.fc.in_features, num_classes)
    else:
        raise ValueError(f"Unsupported model: {model_name}")
    logging.info(f"Initialized {model_name} with {num_classes} classes")
    return model

In [13]:
def load_model(model_path, num_classes, device):
    """
    Load the saved model for inference.
    """
    # Initialize the model with the same architecture used during training
    model = initialize_model("efficientnet", num_classes, pretrained=False)
    
    # Load the saved state dictionary
    model.load_state_dict(torch.load(model_path, map_location=device))
    
    # Move the model to the appropriate device (CPU or GPU)
    model.to(device)
    
    # Set the model to evaluation mode
    model.eval()
    
    return model

In [ ]:
import torchvision.transforms as transforms
from PIL import Image

def preprocess_image(image_path):
    """
    Preprocess the input image for inference.
    """
    # Define the same transformations used during training
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to the input size expected by EfficientNet
        transforms.ToTensor(),          # Convert to tensor
        transforms.Normalize(           # Normalize with the same mean and std used during training
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

def infer(model, image_tensor, device):
    """
    Perform inference on a single image.
    """
    # Move the image tensor to the appropriate device
    image_tensor = image_tensor.to(device)
    
    # Perform inference
    with torch.no_grad():  # Disable gradient calculation
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class
    
    return predicted.item()

In [14]:
# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the saved model
model_path = "models/road_surface_classification.pth"
num_classes = 5  # Replace with the number of classes used during training
model = load_model(model_path, num_classes, device)
"""
# Preprocess the input image
image_path = "path_to_your_image.jpg"
image_tensor = preprocess_image(image_path)

# Perform inference
predicted_class = infer(model, image_tensor, device)
print(f"Predicted class: {predicted_class}")

# Define class names
class_names = ["asphalt", "paving stones", "concrete", "sett", "unpaved"]  # Adjust as needed
"""

c:\Users\drn20\anaconda3\envs\eml\Lib\site-packages\torch\hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to C:\Users\drn20/.cache\torch\hub\torchhub.zip
C:\Users\drn20/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\common.py:13: UserWarning: pytorch_quantization module not found, quantization will no

'\n# Preprocess the input image\nimage_path = "path_to_your_image.jpg"\nimage_tensor = preprocess_image(image_path)\n\n# Perform inference\npredicted_class = infer(model, image_tensor, device)\nprint(f"Predicted class: {predicted_class}")\n\n# Define class names\nclass_names = ["asphalt", "paving stones", "concrete", "sett", "unpaved"]  # Adjust as needed\n'